# Problema de negócio do Big Mart

## Prever a quantidade de vendas por produto

### fonte: https://www.kaggle.com/datasets/shivan118/big-mart-sales-prediction-datasets

### Imports

In [1]:
#importando as bibliotecas que serão utilizadas
import pandas as pd
import numpy as np
import seaborn as sns
import flask
import requests
import pickle

from sklearn.ensemble          import RandomForestRegressor
from sklearn.preprocessing     import OneHotEncoder, StandardScaler
from sklearn.impute            import SimpleImputer
from sklearn.pipeline          import Pipeline
from sklearn.compose           import ColumnTransformer
from sklearn.model_selection   import train_test_split
from sklearn.metrics           import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score

### Loading Data and Visualization

In [2]:
#carregando o dataset
df_train = pd.read_csv('./data/train.csv')
df_test  = pd.read_csv('./data/test.csv')

In [3]:
#exibindo o numero de linhas e colunas do dataset de treino
df_train.shape

(8523, 12)

In [187]:
#exibindo as 5 primeiras linhas do dataset de treino
df_train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [188]:
#exibindo as 5 ultimas linhas do dataset de treino
df_train.tail()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
8518,FDF22,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834
8519,FDS36,8.380,Regular,0.046982,Baking Goods,108.1570,OUT045,2002,NaN,Tier 2,Supermarket Type1,549.2850
8520,NCJ29,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136
8521,FDN46,7.210,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976
8522,DRG01,14.800,Low Fat,0.044878,Soft Drinks,75.4670,OUT046,1997,Small,Tier 1,Supermarket Type1,765.6700


In [189]:
#exibindo informações do dataset de treino
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [190]:
#exibindo informações estatísticas do dataset de treino
df_train.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [191]:
#verificando se possuem valores nulos no dataset de treino
df_train.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

### Clean Data

In [192]:
#função para verificar se linha do Item_Weight possui valor NaN, caso possua verificará em Item_Identifier semelhante se existe valor não NaN, caso existe copiará o valor para a linha

def preencher_peso_faltante(df_train):
    for i, row in df_train.iterrows():
        if pd.isna(row['Item_Weight']):
            item_id = row['Item_Identifier']
            similar_item = df_train[(df_train['Item_Identifier'] == item_id) & (~pd.isna(df_train['Item_Weight']))]
            if len(similar_item) > 0:
                similar_item_weight = similar_item.iloc[0]['Item_Weight']
                df_train.at[i, 'Item_Weight'] = similar_item_weight
    return df_train
df_train = preencher_peso_faltante(df_train)

In [193]:
#analisando e corrigindo inconsistências nos dados
# na coluna Item_Fat_Content encontramos inconsistências pois temos os dados Low Fat, LF e low fat que representam o mesmo dado também os dados Regular e reg que representam o mesmo tipo de dado

# metodo replace para a substituição
df_train = df_train.replace({'LF': 'Low Fat', 'low fat': 'Low Fat', 'reg': 'Regular'})

#na coluna Item_Visibility encontramos valores 0.0 em produtos que tiveram vendas
# Aplicar a função de substituição à coluna 'Item_Visibility' com a média da coluna
def substituir(v):
    if v == 0.000000:
        return 0.066132
    else:
        return v
# Aplicar a função de substituição à coluna 'Item_Visibility'
df_train['Item_Visibility'] = df_train['Item_Visibility'].apply(substituir)

In [194]:
#nos valores nulos de Outlet_Size foram inseridos o dado de valor mais frequente
imputer = SimpleImputer(strategy='most_frequent', missing_values=np.nan) 
imputer = imputer.fit( df_train[['Outlet_Size']] ) 
df_train[['Outlet_Size']] = imputer.transform(df_train[['Outlet_Size']]) 

In [195]:
#verificando novamente se possuem valores nulos
df_train.isnull().sum()

Item_Identifier              0
Item_Weight                  4
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

In [196]:
#apagando valores nulos por linha e conferindo tamanho do DataFrame
df_train = df_train.dropna(axis=0)
df_train.shape

(8519, 12)

In [197]:
#verificando se possuem dados nulos
df_train.isnull().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

### Data Preparation

In [198]:
#excluindo variavel identificadora do produto e variavel resposta 
X = df_train.drop(['Item_Identifier','Item_Outlet_Sales'], axis=1)

#selecionando variável resposta
y = df_train['Item_Outlet_Sales']

In [199]:
#verificando shape das variaveis preditoras e variavel resposta
X.shape, y.shape

((8519, 10), (8519,))

In [200]:
#separando os dados para treinamento e teste
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

In [201]:
#verificando shape dos dados de treinamento e teste
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((6389, 10), (2130, 10), (6389,), (2130,))

In [202]:
#armazenando separadamente as colunas de valores numéricos e categóricos
colunas_numericas = X_train.select_dtypes(exclude='object').columns
colunas_categoricas = X_train.select_dtypes(include='object').columns

In [203]:
#colunas de valores numéricos
colunas_numericas

Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year'],
      dtype='object')

In [204]:
#colunas de valores categóricos
colunas_categoricas

Index(['Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Size',
       'Outlet_Location_Type', 'Outlet_Type'],
      dtype='object')

### Model Training

In [211]:
#transformando variaveis categoricas em numericas
OHE = OneHotEncoder()

#scaling de variaveis numericas
scaler = StandardScaler()

#Regressor
model_rfg = RandomForestRegressor(n_estimators=500,criterion='absolute_error',
                                min_samples_leaf=40,random_state=42, n_jobs=-1)

#transformar as colunas, separando o tipo de transformação com o tipo de variavel
transformer = ColumnTransformer([('cat_cols', OHE, colunas_categoricas),
                              ('num_cols', scaler, colunas_numericas)])

#criando pipeline
pipe = Pipeline([('preprocessing', transformer),
                ('regressor', model_rfg)])


In [212]:
#treinando o modelo
pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('cat_cols', OneHotEncoder(),
                                                  Index(['Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Size',
       'Outlet_Location_Type', 'Outlet_Type'],
      dtype='object')),
                                                 ('num_cols', StandardScaler(),
                                                  Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year'],
      dtype='object'))])),
                ('regressor',
                 RandomForestRegressor(criterion='absolute_error',
                                       min_samples_leaf=40, n_estimators=500,
                                       n_jobs=-1, random_state=42))])

### Predictions

In [207]:
#previsões do modelo em dados de teste
preds = pipe.predict(X_valid)
print('Predictions:',preds[:5])
print('Target:', y_valid.iloc[:5].values)

Predictions: [  79.120343  1025.3513082 2478.3346378 2743.2411444 2254.974717 ]
Target: [  33.9558 1404.1722 1336.2606 3881.614  2093.941 ]


### Performance Metrics

In [208]:
#exibindo o erro do modelo em dados de teste
print('MSE:',mean_squared_error(y_valid, preds))
print('MSE SQRT:',mean_squared_error(y_valid, preds, squared=False))
print('MAE:',mean_absolute_error(y_valid, preds))
print('MAPE:',mean_absolute_percentage_error(y_valid, preds))
print('R2 Score:',r2_score(y_valid, preds))

MSE: 1100915.194261748
MSE SQRT: 1049.2450592029243
MAE: 735.2605746351173
MAPE: 0.5219831550359483
R2 Score: 0.6154447265728762


### Deployment

In [210]:
import pickle

pickle.dump(pipe, open('\\Users\\erico\\Big Mart Sales Prediction Datasets\\deploy_ciclo02\\models\\pipe.pkl', 'wb'))